In [1]:
import pandas as pd

In [2]:
train = pd.read_excel("train.xlsx")
test = pd.read_excel("test.xlsx")
valid = pd.read_excel("valid.xlsx")

In [3]:
train.head()

,comment,target
0,متفق افراد کو جوڑنا سب کو ضرورت کی جگہ پر کر...,neu
1,شاھد بایھی اپ کو ون ڈےکرکٹ میں واپس آجاے,neu
2,سر مجھے بھی ایک دے دیں,neu
3,شاہد بھاہی جنرل راحیل کی طرح شیر ہے,neu
4,پوں پوں آفریدی,neu


In [4]:
test.head()

,comment,target
0,زمان خان نے آوٹ نہ کر کے اچھا کیا باقی سب گیم ...,neu
1,اس غریب کے پاس کوئئ سفارش نہیں تبھی اس بیچارے ...,neu
2,پاکستانی ٹیم میں کمزور پلیئر امام الحق محمد نو...,neu
3,اچھا تھمب نیل میں لڑکی دکھانا لازمی ہے کیا بات ہے,neu
4,آپ میری تجویز پر عمل کریں گے مجھے بڑی امید ھے,neu


In [5]:
valid.head()

,comment,target
0,کچے کیلے یہاں ناروے میں عام دکانوں سے کٹی حالت...,neu
1,کہیں سے بھی ملیں کیلے بس مل جائیں میں ریکوئسٹ ...,neu
2,آپ نے بھی سعود بھائی کی ترکیب سے بنائے تھے جاسمن؟,neu
3,کٹے ہوئے خشک کیلے پکوڑوں کی اس ترکیب کے لیے من...,neu
4,سعود بھائی اگر آپ وعدہ کریں کہ اسی طرح تفصیل س...,neu


In [6]:
print(train.shape,
test.shape,
valid.shape)

(6399, 2) (2001, 2) (1602, 2)


In [7]:
train = train.drop_duplicates()
test = test.drop_duplicates()
valid = valid.drop_duplicates()

In [8]:
print(train.shape,
test.shape,
valid.shape)

(6335, 2) (1984, 2) (1593, 2)


In [9]:
import demoji
import re

from urduhack.preprocessing import replace_urls
from urduhack.preprocessing import replace_numbers
from urduhack.preprocessing import remove_punctuation
from urduhack.preprocessing import remove_accents
from urduhack.preprocessing import remove_english_alphabets
from urduhack.preprocessing import normalize_whitespace
from urduhack.normalization import normalize

def clean_comment(comment):

    comment = demoji.replace(comment,"")

    comment = re.sub(r'@[A-Za-z0-9]+', '', comment)

    comment = re.sub(r'#[A-Za-z0-9]*', '', comment)

    comment = replace_urls(comment,replace_with="")

    comment = replace_numbers(comment,replace_with="")

    comment = remove_accents(comment)

    comment = remove_english_alphabets(comment)

    comment = normalize_whitespace(comment)

    comment = remove_punctuation(comment)

    comment = normalize(comment)

    comment = re.sub(r'\+', '', comment)

    comment = re.sub(r'\=', '', comment)

    return comment

C:\Users\Hizar\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [10]:
for i in train.index:
    a = train['comment'][i]
    b = clean_comment(a)
    train.loc[[i],['comment']] = b

for i in valid.index:
    a = valid['comment'][i]
    b = clean_comment(a)
    valid.loc[[i],['comment']] = b

for i in test.index:
    a = test['comment'][i]
    b = clean_comment(a)
    test.loc[[i],['comment']] = b

In [11]:
train.head()

,comment,target
0,متفق افراد کو جوڑنا سب کو ضرورت کی جگہ پر کرنا...,neu
1,شاھد بایھی اپ کو ون ڈےکرکٹ میں واپس آجاے,neu
2,سر مجھے بھی ایک دے دیں,neu
3,شاہد بھاہی جنرل راحیل کی طرح شیر ہے,neu
4,پوں پوں آفریدی,neu


In [12]:
stop_words = set("""

 آ آئی آئیں آئے آتا آتی آتے آداب آدھ آدھا آدھی آدھے آس آمدید آنا آنسہ آنی آنے
 آپ آگے آہ آہا آیا اب ابھی ابے اتوار ارب اربویں ارے اس اسکا اسکی اسکے اسی اسے اف
 افوہ الاول البتہ الثانی الحرام السلام الف المکرم ان اندر انکا انکی انکے انہوں انہی انہیں اوئے اور
 اوپر اوہو اپ اپنا اپنوں اپنی اپنے اپنےآپ اکبر اکثر اگر اگرچہ اگست اہاہا ایسا ایسی ایسے ایک بائیں
 بار بارے بالکل باوجود باہر بج بجے بخیر برسات بشرطیکہ بعد بعض بغیر بلکہ بن بنا بناؤ بند
 بڑی بھر بھریں بھی بہار بہت بہتر بیگم تاکہ تاہم تب تجھ تجھی تجھے ترا تری تلک تم تمام
 تمہارا تمہاروں تمہاری تمہارے تمہیں تو تک تھا تھی تھیں تھے تہائی تیرا تیری تیرے تین جا جاؤ
 جائیں جائے جاتا جاتی جاتے جانی جانے جب جبکہ جدھر جس جسے جن جناب جنہوں جنہیں جو جہاں جی
 جیسا جیسوں جیسی جیسے جیٹھ حالانکہ حالاں حصہ حضرت خاطر خالی خدا خزاں خواہ خوب خود دائیں درمیان
 دریں دو دوران دوسرا دوسروں دوسری دوشنبہ دوں دکھائیں دگنا دی دیئے دیا دیتا دیتی دیتے دیر دینا دینی
 دینے دیکھو دیں دیے دے ذریعے رکھا رکھتا رکھتی رکھتے رکھنا رکھنی رکھنے رکھو رکھی رکھے رہ رہا
 رہتا رہتی رہتے رہنا رہنی رہنے رہو رہی رہیں رہے ساتھ سامنے ساڑھے سب سبھی سراسر سلام سمیت سوا
 سوائے سکا سکتا سکتے سہ سہی سی سے شام شاید شکریہ صاحب صاحبہ صرف ضرور طرح طرف طور
 علاوہ عین فروری فقط فلاں فی قبل قطا لئے لائی لائے لاتا لاتی لاتے لانا لانی لانے لایا لو
 لوجی لوگوں لگ لگا لگتا لگتی لگی لگیں لگے لہذا لی لیا لیتا لیتی لیتے لیکن لیں لیے
 لے ماسوا مت مجھ مجھی مجھے محترم محترمہ محترمی محض مرا مرحبا مری مرے مزید مس مسز مسٹر مطابق
 مطلق مل منٹ منٹوں مکرمی مگر مگھر مہربانی میرا میروں میری میرے میں نا نزدیک نما نو نومبر
 نہ نہیں نیز نیچے نے و وار واسطے واقعی والا والوں والی والے واہ وجہ ورنہ وعلیکم وغیرہ ولے
 وگرنہ وہ وہاں وہی وہیں ویسا ویسے ویں پاس پایا پر پس پلیز پون پونا پونی پونے پھاگن
 پھر پہ پہر پہلا پہلی پہلے پیر پیچھے چاہئے چاہتے چاہیئے چاہے چلا چلو چلیں چلے چناچہ چند چونکہ
 چوگنی چکی چکیں چکے چہارشنبہ چیت ڈالنا ڈالنی ڈالنے ڈالے کئے کا کاتک کاش کب کبھی کدھر کر
 کرتا کرتی کرتے کرم کرنا کرنے کرو کریں کرے کس کسی کسے کل کم کن کنہیں کو کوئی کون
 کونسا کونسے کچھ کہ کہا کہاں کہہ کہی کہیں کہے کی کیا کیسا کیسے کیونکر کیونکہ کیوں کیے
 کے گئی گئے گا گرما گرمی گنا گو گویا گھنٹا گھنٹوں گھنٹے گی گیا ہائیں ہائے ہاڑ ہاں ہر
 ہرچند ہرگز ہزار ہفتہ ہم ہمارا ہماری ہمارے ہمی ہمیں ہو ہوئی ہوئیں ہوئے ہوا ہوبہو ہوتا ہوتی
 ہوتیں ہوتے ہونا ہونگے ہونی ہونے ہوں ہی ہیلو ہیں ہے یا یات یعنی یک یہ یہاں یہی یہیں


""".split())

def remove_stopwords(text):
    filtered_words = [word for word in text.split() if word not in stop_words]
    return " ".join(filtered_words)

In [13]:
train["comment"] = train.comment.map(remove_stopwords)
valid["comment"] = valid.comment.map(remove_stopwords)
test["comment"] = test.comment.map(remove_stopwords)

In [14]:
train.head()

,comment,target
0,متفق افراد جوڑنا ضرورت جگہ میدان پرفیکٹ فیصلےک...,neu
1,شاھد بایھی ون ڈےکرکٹ واپس آجاے,neu
2,سر,neu
3,شاہد بھاہی جنرل راحیل شیر,neu
4,پوں پوں آفریدی,neu


In [15]:
import urduhack
urduhack.download()
from urduhack.models.lemmatizer import lemmatizer

def lemitizeStr(text):
    lemme_txt = ""
    temp = lemmatizer.lemma_lookup(text)
    for t in temp:
        lemme_txt += t[0] + " "

    return lemme_txt

In [16]:
train["comment"] = train.comment.map(lemitizeStr)
valid["comment"] = valid.comment.map(lemitizeStr)
test["comment"] = test.comment.map(lemitizeStr)

In [17]:
train.head()

,comment,target
0,متفق افراد جوڑنا ضرورت جگہ میدان پرفیکٹ فیصلےک...,neu
1,شاھد بایھی ون ڈےکرکٹ واپس آجاے,neu
2,سر,neu
3,شاہد بھاہی جنرل راحیل شیر,neu
4,پوں پوں آفریدی,neu


In [18]:
train['target'].value_counts()

target
neg    2441
neu    2026
pos    1868
Name: count, dtype: int64

In [19]:
test['target'].value_counts()

target
neg    769
neu    632
pos    583
Name: count, dtype: int64

In [20]:
valid['target'].value_counts()


target
neg    616
neu    504
pos    473
Name: count, dtype: int64

In [21]:
from sklearn.preprocessing import LabelEncoder

class_label_encoder = LabelEncoder()

train['target_encoded'] = class_label_encoder.fit_transform(train['target'])
test['target_encoded'] = class_label_encoder.fit_transform(test['target'])
valid['target_encoded'] = class_label_encoder.fit_transform(valid['target'])

In [22]:
train['target_encoded'].value_counts()

target_encoded
0    2441
1    2026
2    1868
Name: count, dtype: int64

In [23]:
x_train = train["comment"]
x_test = test["comment"]
x_valid = valid["comment"]
y_train = train[["target_encoded"]]
y_test = test[["target_encoded"]]
y_valid = valid[["target_encoded"]]

In [24]:
df = pd.concat([train,valid,test])

In [25]:
df.shape

(9912, 3)

In [26]:
from collections import Counter

def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count

In [27]:
counter = counter_word(df.comment)

In [28]:
num_unique_words = len(counter)
num_unique_words

12362

In [29]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(x_train)

In [30]:
word_index = tokenizer.word_index

In [31]:
len(word_index)

9175

In [32]:
X_train_seq = tokenizer.texts_to_sequences(x_train)
X_valid_seq = tokenizer.texts_to_sequences(x_valid)
X_test_seq = tokenizer.texts_to_sequences(x_test)

In [33]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_length = 20
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding="post")
X_valid_padded = pad_sequences(X_valid_seq, maxlen=max_length, padding="post")
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding="post")
X_train_padded.shape, X_valid_padded.shape, X_test_padded.shape

((6335, 20), (1593, 20), (1984, 20))

In [34]:
print(x_train.iloc[10])

آج ارجنٹینا میچ دیکھ ارجنٹینا جرمنی رکھوں ارجنٹینا برازیل ٹیم فاتح باقی انگلیڈ کھیل مایوس رونی صورتحال سنبھال 


In [35]:
print(X_train_seq[10])

[22, 1256, 3, 37, 1256, 491, 2759, 1256, 794, 2, 524, 149, 4055, 16, 664, 2114, 990, 2115]


In [36]:
print(X_train_padded[10])

[  22 1256    3   37 1256  491 2759 1256  794    2  524  149 4055   16
  664 2114  990 2115    0    0]


In [37]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

encoder = OneHotEncoder(sparse_output=False)

# Fit and transform the data
encoded_array = encoder.fit_transform(train[['target']])

# Create a DataFrame with the encoded data
train_encoded = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out())

# Fit and transform the data
encoded_array = encoder.transform(valid[['target']])  # Use transform instead of fit_transform

# Create a DataFrame with the encoded data
valid_encoded = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out())

# Fit and transform the data
encoded_array = encoder.transform(test[['target']])  # Use transform instead of fit_transform

# Create a DataFrame with the encoded data
test_encoded = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out())


In [38]:
y_train, y_valid, y_test = train_encoded, valid_encoded, test_encoded

In [42]:
from gensim.models import KeyedVectors

word2vec = KeyedVectors.load_word2vec_format('/kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin',  binary=True, limit=10 ** 5)

File not found: /kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin


In [43]:
import os

file_path = '/kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin'
print("File exists:", os.path.exists(file_path))


File exists: False
